In [ ]:
library(ggplot2)
library(cluster)

# Clustering

## Hierarchical Clustering

The algorithm works as follows:

* Put each data point in its own cluster.
* Identify the closest two clusters and combine them into one cluster.
* Repeat the above step till all the data points are in a single cluster.

In [ ]:
levels(iris$Species)

`hclust` requires us to provide the data in the form of a distance matrix. We can do this by using `dist`. 

In [ ]:
d = dist(iris[, 3:4], method = "euclidean")
d

`dist` This function computes and returns the distance matrix computed by using the specified distance measure to compute the distances between the rows of a data matrix.

In general, for a data sample of size M, the distance matrix is an M × M symmetric matrix with M × (M - 1)∕2 distinct elements. Hence for a data sample of size 150 (the number of observations in `iris`), its distance matrix has about 11,000 distinct elements.

In [ ]:
clusters1 <- hclust(d) #defaults to complete linkage method (not covered)
clusters1

![](media/hclust_complete_dist.png)
complete linkage

![](media/hclust_average_dist.png)
average or UPGMA

![](media/hclust_centroid_dist.png)
centroid

In [ ]:
plot(clusters1)
rect.hclust(clusters1 , k = 3, border = 2:6)

In the dendrogram displayed above, each leaf corresponds to one observation. As we move up the tree, observations that are similar to each other are combined into branches, which are themselves fused at a higher height.

The height of the fusion, provided on the vertical axis, indicates the (dis)similarity between two observations. The higher the height of the fusion, the less similar the observations are.

In [ ]:
clusterCut1 <- cutree(clusters1, 3)
clusterCut1

In [ ]:
table(clusterCut1, iris$Species)

It looks like the algorithm successfully classified all the flowers of species setosa into cluster 1, and virginica into cluster 2, but had trouble with versicolor. If you look at the original plot showing the different species, you can understand why:

In [ ]:
ggplot(iris, aes(x=Petal.Length, y=Petal.Width, color=Species)) + geom_point(size = 2)

### Question

Let's see if using a different clustering method will give better results. Try the `average` method on your own (add the argument `method` to the `hclust` function). Plot the respective dendogram. Compare cluster membership among, k=3 clusters, and the members between species.

## K-means Clustering

In k means clustering, we have to specify the number of clusters we want the data to be grouped into. The algorithm randomly assigns each observation to a cluster, and finds the centroid of each cluster. Then, the algorithm iterates through two steps:
* Reassign data points to the cluster whose centroid is closest.
* Calculate new centroid of each cluster.

These two steps are repeated till the within cluster variation cannot be reduced any further. The within cluster variation is calculated as the sum of the euclidean distance between the data points and their respective cluster centroids.

In [ ]:
set.seed(20)
kCluster <- kmeans(iris[, 3:4], 3, nstart = 1)
kCluster

In [ ]:
table(kCluster$cluster, iris$Species)

As we can see, the data belonging to the setosa species got grouped into cluster 3, versicolor into cluster 1, and virginica into cluster 2. The algorithm wrongly classified two data points belonging to versicolor and four data points belonging to virginica.

We can also plot the data to see the clusters:

In [ ]:
irisCluster$cluster <- as.factor(irisCluster$cluster)
ggplot(iris, aes(Petal.Length, Petal.Width, color = irisCluster$cluster)) + geom_point()

In [ ]:
kCluster$cluster

In [ ]:
#reorder the cluster assignment to be congruent with the species membership
c1 = which(kCluster$cluster==1)
c2 = which(kCluster$cluster==2)
c3 = which(kCluster$cluster==3)

v = vector(length=length(kCluster$cluster))
for(i in 1:length(kCluster$cluster)){
    if(i %in% c1 == TRUE) v[i] = 2
    else if(i %in% c2 == TRUE) v[i] = 3
    else v[i] = 1
}
v

ggplot(iris, aes(Petal.Length, Petal.Width, color = Species)) +
    geom_point(alpha = 0.4, size = 3.5) + geom_point(col = v) +
    scale_color_manual(values = c('black', 'red', 'green'))


### Determining optimal number of clusters (k)

In [ ]:
#Elbow Method for finding the optimal number of clusters
# Compute and plot wss for k = 1 to k = 7.
k.max <- 7
wss <- sapply(1:k.max, 
              function(k){kmeans(iris[, 3:4], k, nstart=20)$tot.withinss})
wss
plot(1:k.max, wss,
     type="b", pch = 19, frame = FALSE, 
     xlab="Number of clusters K",
     ylab="Total within-clusters sum of squares")


In [ ]:
#silhouette method
silhouette_score <- function(k){
  km <- kmeans(iris[, 3:4], centers = k, nstart=20)
  ss <- silhouette(km$cluster, dist(iris[, 3:4]))
  mean(ss[, 3])
}
k <- 2:7
avg_sil <- sapply(k, silhouette_score)
plot(k, type='b', avg_sil, xlab='Number of clusters', ylab='Average Silhouette Scores', frame=FALSE)

### Question

Run a K-means analysis for 2 clusters. Compare the table for membership within clusters and membership within species for this analysis to the table from the k=3 analysis. Do you notice anything interesting? 

Food for thought: what biological reason might there be that versicolor and virginica are getting clustered together?